In [15]:

from txt_prep import prepareTxt


from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import dask.array as da
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
dirname='../data'

In [16]:
# Importation des `noms de métiers`
# Création du dico des `noms de métiers`
jobNames = pd.read_csv(os.path.join(dirname,'categories_string.csv'))['0'].to_dict()

# Importation du jeu de données d'entrainement
trainX=dd.read_json(os.path.join(dirname,'train.json'),lines=False).set_index('Id',n_partitions=10).categorize(columns=['gender'])

trainTarget=dd.read_csv(os.path.join(dirname, 'train_label.csv')).set_index('Id').categorize(columns=['Category'])

# Concatenation en un seul DataFrame pour visualiser
trainDF= trainX.assign(Category = trainTarget.Category)

trainDF['description']=trainDF.apply(lambda x:prepareTxt(x['description']),axis=1,meta=(None, 'object'))

In [3]:
for key in jobNames.keys():
    print(key,jobNames[key])

0 pastor
1 model
2 yoga_teacher
3 teacher
4 personal_trainer
5 painter
6 journalist
7 interior_designer
8 surgeon
9 accountant
10 dj
11 physician
12 comedian
13 software_engineer
14 nurse
15 poet
16 dentist
17 chiropractor
18 filmmaker
19 professor
20 photographer
21 rapper
22 psychologist
23 paralegal
24 architect
25 composer
26 attorney
27 dietitian


In [20]:
df=trainDF.assign(prof=1*(trainDF.Category==19)).compute()
Y=df.prof
df.head()
#X_train, X_test, y_train, y_test = train_test_split(df.description, Y)

,description,gender,Category,prof
Id,,,,
0,also ronald asmus policy entrepreneur fellow g...,F,19,1
1,member aicpa wicpa brent graduated university ...,M,9,0
2,dr aster ld teaching research position ben gur...,M,19,1
3,run boutique design studio attending client un...,M,24,0
4,focus cloud security identity access managemen...,M,24,0


In [21]:
# When use pipeline class
tfidf_vect=TfidfVectorizer(min_df=1000,ngram_range=(1, 2))
clf=SGDClassifier(early_stopping=True,n_jobs=-1)
pipeline = Pipeline([('vector',tfidf_vect),("clf",clf)])

In [22]:
# fit
pipeline.fit(df.description,Y)

Pipeline(steps=[('vector', TfidfVectorizer(min_df=1000, ngram_range=(1, 2))),
                ('clf', SGDClassifier(early_stopping=True, n_jobs=-1))])

In [ ]:
cross_val_score(pipeline, df.description,Y,scoring="f1", n_jobs=-1)

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [23]:
#[print(item) for item in tfidf_vect.get_feature_names()]
len(tfidf_vect.get_feature_names())

1502